<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 6: Confidence</h1>


<a name='section_6_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_1">L6.1 Introduction to Confidence Intervals</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_1">L6.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_2">L6.2 z-Scores and Confidence Intervals for Other Distributions</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_2">L6.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_3">L6.3 Another Example and Rules of Significance</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_3">L6.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_4">L6.4 Moments of Distributions and Mapping</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_4">L6.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_5">L6.5 Monte Carlo Integration</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_5">L6.5 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_6">L6.6 Returning to Fitting Supernova Data</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_6">L6.6 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_7">L6.7 Fitting with a More Accurate Model</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_7">L6.7 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_8">L6.8 Fit to Full Cosmological Model</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_8">L6.8 Exercises</a></td>
    </tr>
</table>



<h3>Importing Data (Colab Only)</h3>

If you are in a Google Colab environment, run the cell below to import the data for this notebook. Otherwise, if you have downloaded the course repository, you do not have to run the cell below.

See the source and attribution information below:

>data: data/L04/sn_z_mu_dmu_plow_union2.1.txt <br>
>source: http://supernova.lbl.gov/Union/, https://arxiv.org/abs/1105.3470 <br>
>attribution: The Supernova Cosmology Project, arXiv:1105.3470v1 <br>
>license type: https://arxiv.org/licenses/nonexclusive-distrib/1.0/license.html <br>

In [ ]:
#>>>RUN: L6.0-runcell00

#importing data from git repository

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L04' >> .git/info/sparse-checkout
!git pull origin main

In [ ]:
#>>>RUN: L6.0-runcell01

!pip install lmfit

In [ ]:
#>>>RUN: L6.0-runcell02

import numpy as np                #https://numpy.org/doc/stable/
from scipy import stats           #https://docs.scipy.org/doc/scipy/reference/stats.html
from scipy import optimize as opt #https://docs.scipy.org/doc/scipy/reference/optimize.html
import matplotlib.pyplot as plt   #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
import math                       #https://docs.python.org/3/library/math.html
import csv                        #https://docs.python.org/3/library/csv.html
import lmfit                      #https://lmfit.github.io/lmfit-py/ 

In [ ]:
#>>>RUN: L6.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_6_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.1 Introduction to Confidence Intervals</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_0) | [Exercises](#exercises_6_1) | [Next Section](#section_6_2) |


In [ ]:
#>>>RUN: L6.1-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L06/slides_L06_01.html', width=970, height=550)

In [ ]:
#>>>RUN: L6.1-runcell01

#probability for events within 1 sigma
pM1=stats.norm.cdf(-1)
p1=stats.norm.cdf(1)
print("probability for events within 1 sigma: ", p1-pM1)
print()

#probability for events strictly < 1 sigma upper bound
pM1=stats.norm.cdf(-1)
p1=stats.norm.cdf(1)
print("probability for events strictly < 1 sigma upper bound: ", p1)
print()

#probability for events within 2 sigma
pM2=stats.norm.cdf(-2)
p2=stats.norm.cdf(2)
print("probability for events within 2 sigma: ", p2-pM2)
print()

#probability for events strictly > 2 sigma upper bound
pM2=stats.norm.cdf(-2)
p2=stats.norm.cdf(2)
print("probability for events strictly > 2 sigma upper bound: ", 1-p2)
print()

<a name='exercises_6_1'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_1) | [Next Section](#section_6_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.1.1</span>

Using stats.norm.cdf(), calculate the p value corresponding to $x$ being within $\pm 3\sigma$ of the mean in a normal distribution. Enter your answer as a number (not a percentage) with precision 1e-4.


In [ ]:
#>>>EXERCISE: L6.1.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#YOUR CODE HERE


>#### Follow-up 6.1.1a (ungraded)
>
>What is the delta log likelihood that corresponds to a 3 sigma deviation?

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.1.2</span>

When computing p-values, we often refer to the region of the distribution that we are working with as "left-handed" or "right-handed". For instance, if we wanted to know the probability of a left-handed $3\sigma$ deviation, this would be the area under the curve (the CDF) to the left of the lower $3\sigma$ bound.

What is the probability for a right-handed 5$\sigma$ deviation? What is the probability of ANY deviation greater than 5$\sigma$?

Enter your answer as a list of two numbers (not a percentage) times 1e-7, with precision 1e-3. For instance, an answer of `6.238572e-7` would be reported as `6.238`.
    
Your list should be `[p_right_handed_5sigma, p_any_5sigma]`.

In [ ]:
#>>>EXERCISE: L6.1.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#YOUR CODE HERE


<a name='section_6_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.2 z-Scores and Confidence Intervals for Other Distributions</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_1) | [Exercises](#exercises_6_2) | [Next Section](#section_6_3) |


In [ ]:
#>>>RUN: L6.2-runcell01

from scipy import stats

#Let's do some integrals
p50=stats.norm.cdf(0)
p1=stats.norm.cdf(1)
p2=stats.norm.cdf(2)
p3=stats.norm.cdf(3)
p5=stats.norm.cdf(5)
pM1=stats.norm.cdf(-1)
pM2=stats.norm.cdf(-2)
pM3=stats.norm.cdf(-3)
pM5=stats.norm.cdf(-5)
#print(p50,p1,p2,p3,pM1,pM2,pM3)

#Whats the probability of things fluctuation more that 1\sigma
print(p1-pM1,"within 1 standard deviations")
print(p2-pM2,"within 2 standard deviations")
print(p3-pM3,"within 3 standard deviations")
print(p5-pM5,"within 5 standard deviations")

#Sometimes we only consider 1-sided p-values
print((1.-p1),"to fluctuate above 1 standard deviation")
print((1.-p3),"to fluctuate above 3 standard deviation")
print((1.-p5),"to fluctuate above 5 standard deviation")

In [ ]:
#>>>RUN: L6.2-runcell03

import numpy as np
import matplotlib.pyplot as plt
#code from here(© Eric Kim): https://aegis4048.github.io/comprehensive_confidence_intervals_for_python_developers

#Let's plot a chi2 with 9 degrees of freedom (df)
df = 9 
x = np.linspace(-1, 28, 1000)
y = stats.chi2.pdf(x, df, loc=0, scale=1)

# two-tailed
#Note we will use this function percent point function(ppf), 
#which inverts the cdf and gives a z from a probability
two_right_tail = stats.chi2.ppf(1 - 0.025, df) #left value
two_left_tail  = stats.chi2.ppf(1 - 0.975, df) #right value
print("two tail values:",two_right_tail,two_left_tail)

# one tailed
one_right_tail = stats.chi2.ppf(1 - 0.05, df)
one_left_tail  = stats.chi2.ppf(1 - 0.95, df)
print("one tail values:",one_right_tail,one_left_tail)

plt.style.use('seaborn-whitegrid')
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

for ax in axes:
    ax.plot(x, y, c='black')
    ax.grid(False)
    #ax.xaxis.set_major_formatter(plt.NullFormatter())
    #ax.yaxis.set_major_formatter(plt.NullFormatter())

#now let's fill this from the left
axes[0].fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) <= two_left_tail), facecolor='grey')
axes[0].fill_between(x, 0, y, where=(np.array(x) > two_left_tail) & (np.array(x) < two_right_tail), facecolor='lightgrey')
axes[0].fill_between(x, 0, y, where=(np.array(x) > two_right_tail) & (np.array(x) <= max(x)), facecolor='grey')

axes[1].fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) < one_right_tail), facecolor='lightgrey')
axes[1].fill_between(x, 0, y, where=(np.array(x) > one_right_tail) & (np.array(x) <= max(x)), facecolor='grey')

axes[2].fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) <= one_left_tail), facecolor='grey')
axes[2].fill_between(x, 0, y, where=(np.array(x) > one_left_tail) & (np.array(x) <= max(x)), facecolor='lightgrey')

fig.tight_layout()

In [ ]:
#>>>RUN: L6.2-runcell04

#Let's compute the mean and RMS of a sample from this distribution
df=9
#Let's sample this distribution
y_chi2 = stats.chi2.rvs(size=1000,df=df)
print(y_chi2[0:5]) #print some valued
print("Sampled Mean:",y_chi2.mean(),"Sampled Stddev:",y_chi2.std())

z=1.5 #Let's deviation corresponding to 1.5sigma in a Gauassian
x = np.linspace(-1, 28, 1000)
y = stats.chi2.pdf(x, df, loc=0, scale=1)
two_right_tail = stats.chi2.ppf(1 - stats.norm.cdf(-z), df)
two_left_tail = stats.chi2.ppf(1 - stats.norm.cdf(z), df)

def plotItAll(x,y,y_chi2,z):
    #Now let's plot the filled area using the true pdfs and the assumed variations if it were a Gaussian
    plt.style.use('seaborn-whitegrid')
    #plot distribution
    plt.plot(x, y, c='black',label="$\chi^2$")
    #plot chi2
    plt.hist(y_chi2, histtype='stepfilled', edgecolor='k', alpha=0.4, color='gray', density=True,bins=20,label="events")
    #true values
    plt.fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) <= two_left_tail), facecolor='grey')
    plt.fill_between(x, 0, y, where=(np.array(x) > two_right_tail) & (np.array(x) <= max(x)), facecolor='grey')
    #Mean +/- 1 sigma
    plt.axvline(y_chi2.mean(), c='red',label="mean")
    plt.axvline(y_chi2.mean()+y_chi2.std()*z, c='blue',label="+/-$\sigma_{gaus}$")
    plt.axvline(y_chi2.mean()-y_chi2.std()*z, c='blue')
    plt.xlabel("x")
    plt.ylabel("$\chi^{2}$")
    plt.legend(loc='upper right')
    plt.show()
plotItAll(x,y,y_chi2,z)

<a name='exercises_6_2'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_2) | [Next Section](#section_6_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.2.1</span>

In the last section, we were using the language of "p-values" and standard deviations. Now we will talk about things in terms of "confidence" and "z-score". 

What is the confidence level corresponding to a z-score of 4, for a normal distribution? Enter your answer as a number with precision 1e-6.

In [ ]:
#>>>EXERCISE: L6.2.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.2.2</span>

Again, consider a normal distribution. What is the z-score for an event to be less than some value $x$, where we know the probability that the value is $>x$ is $p=0.40$?

Enter your answer as a number with precision 1e-3.

In [ ]:
#>>>EXERCISE: L6.2.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass


<a name='section_6_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.3 Another Example and Rules of Significance</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_2) | [Exercises](#exercises_6_3) | [Next Section](#section_6_4) |


In [ ]:
#>>>RUN: L6.3-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L06/slides_L06_03.html', width=970, height=550)

In [ ]:
#>>>RUN: L6.3-runcell01

#NOTE: below we define a random seed for identical results between runs.
#These results may appear slightly different from the related video.


#Generate Cauchy data
np.random.seed(6)
y_cauchy = stats.cauchy.rvs(size=10000)


#choose z-score
z=1.0
two_right_tail = stats.cauchy.ppf(1 - stats.norm.cdf(-z))
two_left_tail = stats.cauchy.ppf(1 - stats.norm.cdf(z))
print('left/right-handed values corresponding to z-score:', two_left_tail,two_right_tail)


#print the mean and stdev of the distribution
fig, ax = plt.subplots(figsize=(12, 6))
y_cauchy = stats.cauchy.rvs(size=10000)
print('[mean of cauchy data, stdev of cauchy data]:', y_cauchy.mean(),z*y_cauchy.std())


#plot distribution
ymin = -300
ymax = 300
plt.xlim([ymin,ymax])
x = np.linspace(ymin, ymax, 10000)
y = stats.cauchy.pdf(x, loc=0, scale=1)
plt.plot(x, y, '--', c='black',label="f")
plt.hist(y_cauchy, histtype='stepfilled', edgecolor='k', alpha=0.4, color='gray', density=True,bins=10000)


#plot true-sigma
#plt.fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) <= two_left_tail), facecolor='grey',label='+/- true $\sigma$')
#plt.fill_between(x, 0, y, where=(np.array(x) > two_right_tail) & (np.array(x) <= max(x)), facecolor='grey')
plt.fill_between(x, 0, y, where=(np.array(x) < two_right_tail) & (np.array(x) >= two_left_tail), facecolor='grey',label='+/- true $\sigma$')


plt.axvline(y_cauchy.mean(), c='red',label="mean")
plt.axvline(y_cauchy.mean()+y_cauchy.std()*z, c='blue',label='+/- gaus $\sigma$')
plt.axvline(y_cauchy.mean()-y_cauchy.std()*z, c='blue')
plt.xlabel("x")
plt.ylabel("f(x)")
plt.legend(loc='upper right')
plt.yscale('log')
plt.show()

In [ ]:
#>>>RUN: L6.3-runcell02

#calculate p_value from data
def get_p_value_data(idist, two_left_tail, two_right_tail):
  within_range = []
  for ielem in idist:
    if ielem <= two_right_tail and ielem >= two_left_tail:
      within_range.append(ielem)
  return len(within_range)/len(idist)


#Generate Cauchy data
np.random.seed(6)
y_cauchy = stats.cauchy.rvs(size=10000)

#choose z-score
z=1.0
two_right_tail = stats.cauchy.ppf(1 - stats.norm.cdf(-z))
two_left_tail = stats.cauchy.ppf(1 - stats.norm.cdf(z))

cauchy_p_value = get_p_value_data(y_cauchy, two_left_tail, two_right_tail)
print("p-value from left/right values and data:", cauchy_p_value)
print()

<a name='exercises_6_3'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_3) | [Next Section](#section_6_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.3.1</span>

What is the probability than an event drawn from a Cauchy distribution falls within 1-sigma of the mean? Use the data that we generated above to calculate your probability, and consider using the function `get_p_value_data()` (or write your own).

Enter your answer as a number with precision 1e-3.

In [ ]:
#>>>EXERCISE: L6.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#calculate p_value from data
def get_p_value_data(idist, two_left_tail, two_right_tail):
  within_range = []
  for ielem in idist:
    if ielem <= two_right_tail and ielem >= two_left_tail:
      within_range.append(ielem)
  return len(within_range)/len(idist)

#Generate Cauchy data
np.random.seed(6)
y_cauchy = stats.cauchy.rvs(size=10000)

#choose z-score
z=1.0
two_right_tail = #YOUR CODE HERE
two_left_tail = #YOUR CODE HERE

cauchy_p_value = get_p_value_data(y_cauchy, two_left_tail, two_right_tail)
print("p-value from left/right values and data:", cauchy_p_value)
print()


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.3.2</span>

What is the p-value that corresponds to a z-score of 1 for a Cauchy distribution? Enter your answer as a number with precision 1e-3.

In [ ]:
#>>>EXERCISE: L6.3.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.3.3</span>

Which of the following options best describes a 3$\sigma$ detection:

- background noise
- evidence
- discovery


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.3.4</span>

Which of the following options best describes a 5$\sigma$ detection:

- background noise
- evidence
- discovery


<a name='section_6_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.4 Moments of Distributions and Mapping</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_3) | [Exercises](#exercises_6_4) | [Next Section](#section_6_5) |


In [ ]:
#>>>RUN: L6.4-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L06/slides_L06_04.html', width=970, height=550)

In [ ]:
#>>>RUN: L6.4-runcell01

def raw_moment(X, k, c=0):
    return ((X - c)**k).mean()

def central_moment(X, k):
    if k == 1:
        return X.mean()
    return raw_moment(X=X, k=k, c=X.mean())

def print_moments(X,label):
    print(label+" mean:",central_moment(X,1))
    print(label+" var:" ,central_moment(X,2))
    print(label+" skew:",central_moment(X,3))
    print(label+" kurtosis:",central_moment(X,4))

N=1000000
y_norm = stats.norm.rvs(size=N)
print_moments(y_norm,"normal")

df=9
y_chi2 = stats.chi2.rvs(size=N,df=df)
print_moments(y_chi2,"chi2 df "+str(df))

y_cauchy = stats.cauchy.rvs(size=N)
print_moments(y_cauchy,"cauchy")


In [ ]:
#>>>RUN: L6.4-runcell02

x = stats.lognorm.rvs(s=1, loc=0, scale=5, size=1000, random_state=4)

# plot
def plotdistandq(x,xaxis,evals):
    fig, axes = plt.subplots(1, 2, figsize=(10, 3))
    axes[0].hist(x,density=True)
    axes[0].plot(xaxis,evals)
    stats.probplot(x, dist=stats.norm, plot=axes[1])
    fig.tight_layout()

xaxis=np.linspace(0,80, 1000)
evals=stats.lognorm.pdf(xaxis,s=1, loc=0, scale=5)
plotdistandq(x,xaxis,evals)
# box-cox transform
xt, lmbda = stats.boxcox(x)

#now plot Gaussian
gxaxis = np.linspace(-2, 5, 1000)
gevals = stats.norm.pdf(gxaxis,xt.mean(),1) 
plotdistandq(xt,gxaxis,gevals)

stats.probplot(xt, dist=stats.norm, plot=axes[1])
fig.tight_layout()

In [ ]:
#>>>RUN: L6.4-runcell03

x = np.abs(stats.cauchy.rvs(size=10000))

xaxis=np.linspace(0,80, 1000)
evals=np.abs(stats.cauchy.pdf(xaxis))
plotdistandq(x,xaxis,evals)
# box-cox transform
xt, lmbda = stats.boxcox(x)

#now plot Gaussian
gxaxis = np.linspace(-2, 5, 1000)
gevals = stats.norm.pdf(gxaxis,xt.mean(),1) 
plotdistandq(xt,gxaxis,gevals)

stats.probplot(xt, dist=stats.norm, plot=axes[1])
fig.tight_layout()

print("mean:",xt.mean())

<a name='exercises_6_4'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_4) | [Next Section](#section_6_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.4.1</span>

Take a $\chi^2$ distribution with 1 degree of freedom, which is very asymmetric, and map it onto a Gaussian. What is the mean of this distribution?

Complete the code below, which uses a random seed to ensure your answer will match ours. Enter your answer as a number with precision 1e-3.


In [ ]:
#>>>EXERCISE: L6.4.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

np.random.seed(10)
x = stats.chi2.rvs(size=10000,df=1)

#YOUR CODE HERE

<a name='section_6_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.5 Monte Carlo Integration</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_4) | [Exercises](#exercises_6_5) | [Next Section](#section_6_6) |


In [ ]:
#>>>RUN: L6.5-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L06/slides_L06_05.html', width=970, height=550)

In [ ]:
#>>>RUN: L6.5-runcell01

import math

#First let's just compute the area of a quarter circle with radius 1
def quarterarea(iN):
    area=0
    lXin = np.array([])
    lYin = np.array([])
    lXout = np.array([])
    lYout = np.array([])
    for i0 in range(iN):
        #Sample X and Y
        pX = np.random.uniform(0,1)
        pY = np.random.uniform(0,1)
        #Check if its radius is in 1
        if math.sqrt(pX**2+pY**2) < 1:
            lXin = np.append(lXin,pX)
            lYin = np.append(lYin,pY)
            area += 1 # count it
        else:
            lXout = np.append(lXout,pX)
            lYout = np.append(lYout,pY)
    return (float(area)/float(iN)),lXin,lYin,lXout,lYout

#sample points
lN=1000
#lN=100000
a,lXin,lYin,lXout,lYout=quarterarea(lN)
print("Pi (4*area):",a*4,"+/-",4*a/math.sqrt(lN)) #gotta put an uncertainty
plt.plot(lXin,lYin,marker='.',linestyle = 'None')
plt.plot(lXout,lYout,marker='.',linestyle = 'None')
plt.show()

In [ ]:
#>>>RUN: L6.5-runcell02

from scipy import optimize as opt 

#use this random seed
np.random.seed(10)

#Now let's consider integrating some random function
def f(x):
    return x**4 + 3*(x-2)**3 - 15*(x)**2 + 1

#Now let's multiply it by -1 to make the range calculation fast
def fneg(x):
    return -1*(x**4 + 3*(x-2)**3 - 15*(x)**2 + 1)

#First thing is to define a range in x
xmin=-6
xmax=3
x = np.linspace(xmin, xmax, 100)
#plt.plot(x, f(x));

#Now we need to find a range in y
sol=opt.minimize_scalar(f,bounds=(xmin, xmax), method='Brent')
ymin=sol.fun
#y-max is to get the minimum of negative f
sol=opt.minimize_scalar(fneg,bounds=(xmin, xmax), method='Brent')
ymax=-1*sol.fun
print('[ymin,ymax]:', ymin, ymax)

lN=100000
#now, let's sample a 2D grid y-min and y-max and compute the integral
lXin = np.array([])
lYin = np.array([])
lXout = np.array([])
lYout = np.array([])
for i0 in range(lN):
    #Try a uniform distribution
    pX = abs(xmax-xmin)*np.random.uniform(0,1)+xmin
    pY = abs(ymax-ymin)*np.random.uniform(0,1)+ymin
    #Try a normal distribution
    #pX = abs(xmax-xmin)*np.random.normal(0,1)+xmin
    #pY = abs(ymax-ymin)*np.random.normal(0,1)+ymin
    pYMin = f(pX)
    if pY < pYMin:
        lXin = np.append(lXin,pX)
        lYin = np.append(lYin,pY)
    else:
        lXout = np.append(lXout,pX)
        lYout = np.append(lYout,pY)


plt.plot(lXin,lYin,marker='.',linestyle = 'None', color='orange')
plt.plot(lXout,lYout,marker='.',linestyle = 'None', color='green')
plt.axvline(sol.x, c='red', lw=3)
plt.plot(x, f(x), 'b-', lw=3)
#plt.ylim(-800,0)
#plt.xlim(-6,3)
plt.show();

In [ ]:
#>>>RUN: L6.5-runcell03

_,bins,_=plt.hist(lXin,bins=10,alpha=0.5,density=True, label='below')
plt.hist(lXout,alpha=0.5,bins=bins,density=True, label='above')
#plt.xlim(-6,3)
plt.legend(loc=1)
plt.show();

print("number below:",len(lXin),"number above:",len(lXout))
#Note, these numbers will be slightly different for each run
#due to random generation of data

<a name='exercises_6_5'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_5) | [Next Section](#section_6_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.5.1</span>

Compute the fraction of events above the line corresponding to the function that we defined previously:

<pre>
def f(x):
    return x**4 + 3*(x-2)**3 - 15*(x)**2 + 1
</pre>

Specifically, sample `100000` points from a uniform distribution over the x and y ranges specified above (i.e., $-6 < x < 3$ and $f(-6) < y < f(3)$ to compute this fraction.

Enter your answer as a number with precision 1e-3.

Hint: Use the `lXout` value from `L6.5-runcell02`. This means the solution is only 3 lines.

In [ ]:
#>>>EXERCISE: L6.5.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#YOUR CODE HERE
#Note, if you are computing the distributions again, use the following random seed
np.random.seed(10)
pass


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.5.2</span>

How would you compute the integral of this function over the range `[-6,3]`? Select the correct answer below:

- The integral is equal to the number of points above the line defined by our function.
- The integral is equal to the number of points between zero and the line defined by our function.
- The integral is equal to the fraction of points that we calculated, times the area of the region that we defined.
- The integral is equal to the fraction of points between zero and our function, times the area of the region where the simulation is carried out, which must use `0` as an upper bound.


>#### Follow-up 6.5.2a (ungraded)
>
>Calculate the integral numerically. Remember to correctly apply a negative sign, if applicable.
>
>How does your numerically calculated value compare to the expected value? You can explicitly solve the integral mathematically, or use a built-in `scipy` integration method to compare to your Monte Carlo integration.

In [ ]:
#>>>FOLLOW-UP: L6.5.2a
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#YOUR CODE HERE
pass


<a name='section_6_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.6 Returning to Fitting Supernova Data</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_5) | [Exercises](#exercises_6_6) | [Next Section](#section_6_7) |


In [ ]:
#>>>RUN: L6.6-runcell01
import math
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy import stats

#Let's try to understand how good the fits we made in last Lesson are, let's load the supernova data again
label='data/L04/sn_z_mu_dmu_plow_union2.1.txt'

def distanceconv(iMu):
    power=iMu/5+1
    return 10**power

def distanceconverr(iMu,iMuErr):
    power=iMu/5+1
    const=math.log(10)/5.
    return const*(10**power)*iMuErr

#only reads data up to z=0.1.
def load(iLabel,iMaxZ=0.1):
    redshift=np.array([])
    distance=np.array([])
    distance_err=np.array([])
    with open(iLabel,'r') as csvfile:
        plots = csv.reader(csvfile, delimiter='\t')
        for row in plots:
            if float(row[1]) > iMaxZ:
                continue
            redshift = np.append(redshift,float(row[1]))
            distance = np.append(distance,distanceconv(float(row[2])))
            distance_err = np.append(distance_err,distanceconverr(float(row[2]),float(row[3])))
    return redshift,distance,distance_err  
        
#Now let's run the regression again
def variance(isamples):
    mean=isamples.mean()
    n=len(isamples)
    tot=0
    for pVal in isamples:
        tot+=(pVal-mean)**2
    return tot/n

def covariance(ixs,iys):
    meanx=ixs.mean()
    meany=iys.mean()
    n=len(ixs)
    tot=0
    for i0 in range(len(ixs)):
        tot+=(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/n

def linear(ix,ia,ib):
    return ia*ix+ib

redshift,distance,distance_err=load(label)
var=variance(redshift)
cov=covariance(redshift,distance)
A=cov/var
const=distance.mean()-A*redshift.mean()
xvals = np.linspace(0,0.1,100)
yvals = []
for pX in xvals:
    yvals.append(linear(pX,A,const))

plt.plot(xvals,yvals)
plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.xlabel("z(redshift)")
plt.ylabel("distance(pc)")
plt.show()

In [ ]:
#>>>RUN: L6.6-runcell02

def residualsComp(redshift,distance,distance_err):
    #Compute residuals
    residuals=np.array([])
    for i0 in range(len(redshift)):
        pResid=linear(redshift[i0],A,const)-distance[i0]
        residuals = np.append(residuals,pResid/distance_err[i0])
    
    #Make a histogram
    y0, bin_edges = np.histogram(residuals, bins=30)
    bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
    norm0=len(residuals)*(bin_edges[-1]-bin_edges[0])/30.
    plt.errorbar(bin_centers,y0/norm0,yerr=y0**0.5/norm0,marker='.',drawstyle = 'steps-mid')
    
    #Plot a Gaussian
    k=np.arange(bin_edges[0],bin_edges[-1],0.05)
    normal=stats.norm.pdf(k,0,1)
    #First let's look at the moments 
    normalpoints=stats.norm.rvs(0,1,1000)
    print_moments(residuals,"residuals")
    print_moments(normalpoints,"normal distribution")

    #Now let's plot it
    plt.plot(k,normal,'o-')
    plt.xlabel("number of successes")
    plt.ylabel("probability")
    plt.show()
    return residuals

residuals=residualsComp(redshift,distance,distance_err)

In [ ]:
#>>>RUN: L6.6-runcell03

#now let's look at the chi2
chi2=np.sum(residuals**2)

print("Total chi2:",chi2,"NDOF",len(residuals)-2)
print("Normalized chi2:",chi2/(len(residuals)-2))
print("Probability of chi2:",1-stats.chi2.cdf(chi2,(len(residuals)-2)))
print()

#Let's plot it for good measure too
x = np.linspace(0,len(residuals)*2)
chi2d=stats.chi2.pdf(x,len(residuals-2)) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2, c='red')
plt.legend(loc='lower right')

ndof=(len(residuals)-2)
chi2ppf0=stats.chi2.ppf(0.5,ndof)
chi2ppf1=stats.chi2.ppf(0.15,ndof)
chi2ppf2=stats.chi2.ppf(0.85,ndof)
#chi2ppf1=stats.chi2.ppf(0.025,ndof)
#chi2ppf2=stats.chi2.ppf(1-0.025,ndof)
print("Central Vvalue",chi2ppf0)
print("Sigma Low",chi2ppf1-chi2ppf0)
print("Sigma High",chi2ppf2-chi2ppf0)

plt.axvline(chi2ppf1, c='blue')
plt.axvline(chi2ppf2, c='blue')
plt.show()

<a name='exercises_6_6'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_6) | [Next Section](#section_6_7) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.6.1</span>

Fill in the blank: Anything above ___ for chi squared probability is a sign of a good fit. Enter your answer as a number with precision 1e-2.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.6.2</span>

Fill in the blank: For a normal distribution, a good fit has a normalized chi squared value near ___ . Enter your answer as a number with precision 1e-2.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.6.3</span>

Ok, now let's repeat the linear fit above without a cut on the redshift. Let's run our full slew of metrics on it like we did before. Complete the code below and analyze the output.

What is the value of the normalized (or reduced) chi-square metric? Is this a good fit or a bad fit? Enter your answer as a number with precision 1e-2.

In [ ]:
#>>>RUN: L6.6.3

#run regression
redshift,distance,distance_err=load(label,10000)
var=variance(redshift)
cov=covariance(redshift,distance)
A=cov/var
const=distance.mean()-A*redshift.mean()
xvals = np.linspace(0,1.4,100)
yvals = []
for pX in xvals:
    yvals.append(linear(pX,A,const))

#plot it
plt.plot(xvals,yvals)
plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.xlabel("z(redshift)")
plt.ylabel("distance(pc)")
plt.show()

residuals=residualsComp(redshift,distance,distance_err)
#now let's look at the chi2
#YOUR CODE HERE

<a name='section_6_7'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.7 Fitting with a More Accurate Model</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_6) | [Exercises](#exercises_6_7) | [Next Section](#section_6_8) |


In [ ]:
#>>>RUN: L6.7-runcell02

#We are not going to plot the fit first, let's just use our barrage of statistics to check if its ok
def hubble(z,Om):
    pVal=Om*(1+z)**3+(1.-Om)
    return np.sqrt(pVal)

def lumidistance(z,h0,Om):
    integral=0
    nint=100
    for i0 in range(nint):
        zp=z*float(i0)/100.
        dz=z/float(nint)
        pVal=1./(1e-5+hubble(zp,Om))
        integral += pVal*dz
    d=(1.+z)*integral*(1e6*3e5/h0)
    return d

print("test Lumi",lumidistance(1,70,0.3))


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.7.1</span>

How would more dark energy change the luminosity distance over redshift? You can approach this problem mathematically, or simply change the apppropriate parameter in the function `lumidistance()` to compute the luminosity distance at different values.

Based on your observations, if there is more dark energy, the luminosity distance should:

- get larger
- get smaller
- stay the same


In [ ]:
#>>>EXERCISE: L6.7.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass

<a name='section_6_8'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.8 Fit to Full Cosmological Model</h2>     

| [Top](#section_6_0) | [Previous Section](#section_6_7) | [Exercises](#exercises_6_8) |


In [ ]:
#>>>RUN: L6.8-runcell01

import lmfit

model  = lmfit.Model(lumidistance)
p = model.make_params(h0=70,Om=0.2)
result = model.fit(data=distance, params=p, z=redshift, weights=1./distance_err)
lmfit.report_fit(result)
plt.figure()
result.plot()

In [ ]:
#>>>RUN: L6.8-runcell02
from scipy import optimize as opt 

def loglike(x):
    lTot=0
    for i0 in range(len(redshift)):
        xtest=lumidistance(redshift[i0],x[0],x[1])
        #lTot = lTot+(distance[i0]-xtest)**2
        lTot = lTot+((1./distance_err[i0])**2)*(distance[i0]-xtest)**2
    return lTot #*0.5 The above is 2 times loglike

def residuals(x):
    residuals=np.array([])
    for i0 in range(len(redshift)):
        pResid=lumidistance(redshift[i0],sol.x[0],sol.x[1])-distance[i0]
        residuals = np.append(residuals,pResid/distance_err[i0])
    return residuals


x0 = np.array([60.,0.2])
ps = [x0]
bnds = ((0, 1000), (0, 1.0))
sol=opt.minimize(loglike, x0,bounds=bnds, tol=1e-6)
print(sol)
residuals=residuals(sol.x)
print_moments(residuals,"residuals")
chi2=np.sum(residuals**2)
print("Total chi2:",chi2,"NDOF",len(residuals)-2)
print("Normalized chi2:",chi2/(len(residuals)-2))
print("Probability of chi2:",1-stats.chi2.cdf(chi2,(len(residuals)-2)))

#Let's plot it for good measure too
x = np.linspace(0,len(residuals)*2)
chi2d=stats.chi2.pdf(x,len(residuals-2)) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2, c='red')
plt.legend(loc='lower right')
plt.show();

In [ ]:
#>>>RUN: L6.8-runcell03

#Plot it against the data
xvals = np.linspace(0,1.4,100)
yvals = []
for pX in xvals:
    yvals.append(lumidistance(pX,sol.x[0],sol.x[1]))

plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.plot(xvals,yvals)
plt.show()

#Histogram the residuals
y0, bin_edges = np.histogram(residuals, bins=30)
bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
norm0=len(residuals)*(bin_edges[-1]-bin_edges[0])/30.
plt.errorbar(bin_centers,y0/norm0,yerr=y0**0.5/norm0,marker='.',drawstyle = 'steps-mid')
k=np.arange(bin_edges[0],bin_edges[-1],0.05)
normal=stats.norm.pdf(k,0,1)
plt.plot(k,normal,'o-')
plt.show()

x = np.linspace(len(residuals)*0.5,len(residuals)*1.5)
chi2d=stats.chi2.pdf(x,len(residuals-2)) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2, c='red')
plt.legend(loc='lower right')
plt.show()




<a name='exercises_6_8'></a>   

| [Top](#section_6_0) | [Restart Section](#section_6_8) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.8.1</span>

So, given the Friedmann equations, we can add back the curvature term:

$$ d(z) = ct^{\prime} = (1+z)ct = (1+z)\frac{c}{h_{0}}\int_{0}^{z} \frac{dz^{\prime}}{\sqrt{\Omega_{M}\left(1+z^{\prime}\right)^{3} + \Omega_{\kappa}\left(1+z^{\prime}\right)^{2}+ 1-\Omega_{M}-\Omega_{\kappa}}}$$

Adjust the `lumidistance()` function to fit for curvature. What is the value of $\Omega_{\kappa}$ if we perform a new fit to the data? What do you think this is implying? 

Enter your answer as a number with precision 1e-2.


In [ ]:
#>>>EXERCISE: L6.8.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def hubble_curve(z,Om,OmK):
    return #YOUR CODE HERE

def lumidistance_curve(z,h0,Om,OmK):
    return #YOUR CODE HERE


model  = lmfit.Model(lumidistance_curve)
p = model.make_params(h0=70,Om=0.2,OmK=0.0)
result = model.fit(data=distance, params=p, z=redshift, weights=1./distance_err)
lmfit.report_fit(result)
result.plot();
